In [74]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [75]:
name = 'mnist-784-euclidean'
nameFull = 'mnist-784-euclidean-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (60000, 784)
testDataset :  (10000, 784)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [76]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = datasetImages.shape[1]
    time_start = perf_counter()
    index = indexMethod(d, M)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexLSH, datasetTrainImages)

Took 0.589 seconds


In [77]:
print(indexedStruct)

<faiss.swigfaiss.IndexLSH; proxy of <Swig Object of type 'faiss::IndexLSH *' at 0x000001D31CE13E40> >


In [78]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [79]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 2.175 seconds


In [80]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n', 'max : ', max,)

In [81]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [82]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [83]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [84]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

53843 || 53843
90.0 || 676.584
38620 || 38620
91.0 || 793.9868
16186 || 16186
93.0 || 862.6766
21518 || 27059
98.0 || 864.5039
15260 || 47003
99.0 || 894.7
31073 || 14563
105.0 || 909.7043
36395 || 44566
106.0 || 917.6323
14563 || 15260
108.0 || 921.6241
38504 || 40368
109.0 || 922.147
9724 || 36395
110.0 || 943.4972


In [85]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

1.0000
1.0000
1.0000
